### Лабораторная работа №6

Для произвольного набора данных, предназначенного для классификации текстов, решите задачу классификации текста двумя способами:

1.   Способ 1. На основе CountVectorizer или TfidfVectorizer.
2.   Способ 2. На основе моделей word2vec или Glove или fastText.
3.   Сравните качество полученных моделей.

Для поиска наборов данных в поисковой системе можно использовать ключевые слова "datasets for text classification".

In [47]:
import re
import pandas as pd
import numpy as np
from typing import Dict, Tuple
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.svm import LinearSVC

In [48]:
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/parallels/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
from sklearn.datasets import fetch_20newsgroups

categories = ["rec.sport.baseball", "rec.autos", "sci.space", "talk.politics.guns"]
newsgroups = fetch_20newsgroups(subset='train', categories=categories)
train = newsgroups['data']

### <u>CountVectorizer

In [50]:
vocabVect = CountVectorizer()
vocabVect.fit(train)
corpusVocab = vocabVect.vocabulary_
print('Количество сформированных признаков - {}'.format(len(corpusVocab)))

Количество сформированных признаков - 36320


In [51]:
for i in list(corpusVocab)[1:10]:
    print('{}={}'.format(i, corpusVocab[i]))

looper=21139
cco=9458
caltech=9116
edu=13479
mark=21794
subject=31556
re=27442
command=10375
loss=21171


In [52]:
test_features = vocabVect.transform(train)

In [53]:
test_features

<2330x36320 sparse matrix of type '<class 'numpy.int64'>'
	with 373978 stored elements in Compressed Sparse Row format>

In [54]:
test_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [55]:
# Размер нулевой строки
len(test_features.todense()[0].getA1())

36320

In [56]:
# Непустые значения нулевой строки
[i for i in test_features.todense()[0].getA1() if i>0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 3,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 5,
 1,
 1,
 2,
 1,
 1,
 6,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 3,
 1,
 2,
 1,
 6,
 17,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 3,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1]

In [57]:
vocabVect.get_feature_names()[10000:10020]

['clarke',
 'clarkson',
 'clarku',
 'clas',
 'clash',
 'class',
 'classes',
 'classic',
 'classical',
 'classification',
 'classified',
 'classify',
 'classroom',
 'claudio',
 'clause',
 'clauses',
 'claw',
 'clay',
 'clayco',
 'claypigeon']

Решение задачи анализа тональности текста на основе модели "мешка слов"

С использованием кросс-валидации попробуем применить к корпусу текстов различные варианты векторизации и классификации.

In [58]:
def VectorizeAndClassify(vectorizers_list, classifiers_list):
    for v in vectorizers_list:
        for c in classifiers_list:
            pipeline1 = Pipeline([("vectorizer", v), ("classifier", c)])
            score = cross_val_score(pipeline1, newsgroups['data'], newsgroups['target'], scoring='accuracy', cv=3).mean()
            print('Векторизация - {}'.format(v))
            print('Модель для классификации - {}'.format(c))
            print('Accuracy = {}'.format(score))
            print('===========================')

In [59]:
vectorizers_list = [CountVectorizer(vocabulary = corpusVocab), TfidfVectorizer(vocabulary = corpusVocab)]
classifiers_list = [LogisticRegression(C=3.0), LinearSVC(), KNeighborsClassifier()]
VectorizeAndClassify(vectorizers_list, classifiers_list)

/home/parallels/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/parallels/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

Векторизация - CountVectorizer(vocabulary={'00': 0, '000': 1, '0000': 2, '00000': 3,
                            '000000': 4, '000021': 5, '000062david42': 6,
                            '000152': 7, '00041032': 8, '0004136': 9,
                            '0004246': 10, '0004422': 11, '00044513': 12,
                            '0004847546': 13, '0005': 14, '0005111312': 15,
                            '0005111312na3em': 16, '000601': 17, '000710': 18,
                            '00090711': 19, '000mi': 20, '000miles': 21,
                            '000s': 22, '000th': 23, '001': 24, '0010': 25,
                            '0012': 26, '001211': 27, '001319': 28,
                            '001428': 29, ...})
Модель для классификации - LogisticRegression(C=3.0)
Accuracy = 0.9622308243442265
Векторизация - CountVectorizer(vocabulary={'00': 0, '000': 1, '0000': 2, '00000': 3,
                            '000000': 4, '000021': 5, '000062david42': 6,
                            '000152

### word2vec

In [60]:
corpus = []
stop_words = stopwords.words('english')
tok = WordPunctTokenizer()
for line in train:
    line1 = line.strip().lower()
    line1 = re.sub("[^a-zA-Z]"," ", line1)
    text_tok = tok.tokenize(line1)
    text_tok1 = [w for w in text_tok if not w in stop_words]
    corpus.append(text_tok1)

In [61]:
corpus[:5]

[['looper',
  'cco',
  'caltech',
  'edu',
  'mark',
  'looper',
  'subject',
  'command',
  'loss',
  'timer',
  'galileo',
  'update',
  'organization',
  'california',
  'institute',
  'technology',
  'pasadena',
  'lines',
  'nntp',
  'posting',
  'host',
  'sandman',
  'caltech',
  'edu',
  'keywords',
  'galileo',
  'jpl',
  'prb',
  'access',
  'digex',
  'com',
  'pat',
  'writes',
  'galileo',
  'hga',
  'stuck',
  'hga',
  'left',
  'closed',
  'galileo',
  'venus',
  'flyby',
  'hga',
  'pointed',
  'att',
  'sun',
  'near',
  'venus',
  'would',
  'cook',
  'foci',
  'elements',
  'question',
  'galileo',
  'course',
  'manuevers',
  'designed',
  'hga',
  'ever',
  'sun',
  'point',
  'hga',
  'reflective',
  'wavelengths',
  'might',
  'cook',
  'focal',
  'elements',
  'figure',
  'good',
  'scales',
  'problem',
  'antenna',
  'could',
  'exposed',
  'venus',
  'level',
  'sunlight',
  'lest',
  'like',
  'icarus',
  'wings',
  'melt',
  'think',
  'glues',
  'well',
  

In [62]:
# количество текстов в корпусе не изменилось и соответствует целевому признаку
assert len(train)==len(corpus)

In [63]:
from gensim.models import word2vec

%time model = word2vec.Word2Vec(corpus)

CPU times: user 6.43 s, sys: 86.8 ms, total: 6.51 s
Wall time: 3.63 s


In [64]:
print(model.wv.most_similar(positive=['find'], topn=5))

[('wrong', 0.9927355647087097), ('going', 0.9916925430297852), ('far', 0.9904637932777405), ('pretty', 0.9900212287902832), ('still', 0.9899173974990845)]


In [65]:
def sentiment(v, c):
    model = Pipeline(
        [("vectorizer", v), 
         ("classifier", c)])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print_accuracy_score_for_classes(y_test, y_pred)

In [66]:
class EmbeddingVectorizer(object):
    '''
    Для текста усредним вектора входящих в него слов
    '''
    def __init__(self, model):
        self.model = model
        self.size = model.vector_size

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([np.mean(
            [self.model[w] for w in words if w in self.model] 
            or [np.zeros(self.size)], axis=0)
            for words in X])

In [67]:
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res

def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

In [68]:
boundary = 700
X_train = corpus[:boundary] 
X_test = corpus[boundary:]
y_train = newsgroups['target'][:boundary]
y_test = newsgroups['target'][boundary:]

In [69]:
sentiment(EmbeddingVectorizer(model.wv), LogisticRegression(C=5.0))

/home/parallels/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Метка 	 Accuracy
0 	 0.8329113924050633
1 	 0.7775229357798165
2 	 0.8672985781990521
3 	 0.8222811671087533
